In [1]:
import utils as utl
import faiss
import numpy as np

# Loading data

In [2]:
vectors_file="../.data/vectors.pkl"
# 'text-embedding-ada-002', 'all-MiniLM-L6-v2'
model="all-MiniLM-L6-v2"
embeddings = utl.load_pickle(vectors_file)
vectors = embeddings[model]
print(f" pickle file returned {len(vectors)} vectors for model '{model}'")

 pickle file returned 1363 vectors for model 'all-MiniLM-L6-v2'


# Format data
vectors are `{hash:vector}` map

In [4]:
np_vectors = np.array(list(vectors.values()))
np_vectors.shape

(1363, 384)

# Indexing

In [8]:
dimension = np_vectors.shape[1]
faiss_index = faiss.IndexFlatL2(dimension)

In [9]:
%time
faiss_index.add(np_vectors)

CPU times: total: 0 ns
Wall time: 0 ns


In [10]:
faiss.write_index(faiss_index,"../.data/faiss.index")

# Query

In [13]:
sentence = "A smart sensor with a modern wireless protocol, that can sense the environment without consuming much power"
query_vector = utl.get_one_embedding(sentence,model)
np_query_vector = np.array(query_vector).reshape(1,-1)
np_query_vector.shape

 => 1 embeddings in 135 ms 


(1, 384)

## Search

In [14]:
top_k = 5
distances, indices = faiss_index.search(np_query_vector, top_k)

In [16]:
result = (distances, indices)
result

(array([[1.0959078, 1.120291 , 1.1375129, 1.1995885, 1.2544465]],
       dtype=float32),
 array([[1009,  635,  978,  916,  206]], dtype=int64))

In [19]:
hashes_list = list(vectors.keys())
hashes = [hashes_list[index] for index in indices[0] ]
hashes

['952744b4', 'ea3de306', '23b7911d', '0bf20e45', 'b5d24268']